# Numerical differentiation

In this tutorial, you will learn how to numerically differentiate functions with
optimagic.

In [ ]:
import numpy as np
import optimagic as om
import pandas as pd

## Basic usage of `first_derivative`

In [ ]:
def fun(params):
    return params @ params

In [ ]:
fd = om.first_derivative(
    func=fun,
    params=np.arange(5),
)

fd.derivative

## Basic usage of `second_derivative`

In [ ]:
sd = om.second_derivative(
    func=fun,
    params=np.arange(5),
)

sd.derivative.round(3)

## You can parallelize

In [ ]:
fd = om.first_derivative(
    func=fun,
    params=np.arange(5),
    n_cores=4,
)

fd.derivative

In [ ]:
sd = om.second_derivative(
    func=fun,
    params=np.arange(5),
    n_cores=4,
)

sd.derivative.round(3)

## `params` do not have to be vectors

In optimagic, params can be arbitrary [pytrees](https://jax.readthedocs.io/en/latest/pytrees.html). Examples are (nested) dictionaries of numbers, arrays and pandas objects. 

In [ ]:
def dict_fun(params):
    return params["a"] ** 2 + params["b"] ** 2 + (params["c"] ** 2).sum()

In [ ]:
fd = om.first_derivative(
    func=dict_fun,
    params={"a": 0, "b": 1, "c": pd.Series([2, 3, 4])},
)

fd.derivative

### Description of the output

> Note. Understanding the output of the first and second derivative requires terminolgy
> of pytrees. Please refer to the
> [JAX documentation of pytrees](https://jax.readthedocs.io/en/latest/pytrees.html).

The output tree of `first_derivative` has the same structure as the params tree.
Equivalent to the 1-d numpy array case, where the gradient is a vector of shape
`(len(params),)`. If, however, the params tree contains non-scalar entries like
`numpy.ndarray`'s, `pandas.Series`', or `pandas.DataFrame`'s, the output is not expanded
but a block is created instead. In the above example, the entry `params["c"]` is a
`pandas.Series` with 3 entries. Thus, the first derivative output contains the
corresponding 3x1-block of the gradient at the position `["c"]`:

In [ ]:
fd.derivative["c"]

In [ ]:
sd = om.second_derivative(
    func=dict_fun,
    params={"a": 0, "b": 1, "c": pd.Series([2, 3, 4])},
)

sd.derivative

### Description of the output

> Note. Understanding the output of the first and second derivative requires terminolgy
> of pytrees. Please refer to the
> [JAX documentation of pytrees](https://jax.readthedocs.io/en/latest/pytrees.html).

The output of `second_derivative` when using a general pytrees looks more complex but
is easy once we remember that the second derivative is equivalent to applying the first
derivative twice.

The output tree is a product of the params tree with itself. This is equivalent to the
1-d numpy array case, where the hessian is a matrix of shape
`(len(params), len(params))`. If, however, the params tree contains non-scalar entries
like `numpy.ndarray`'s, `pandas.Series`', or `pandas.DataFrame`'s, the output is not
expanded but a block is created instead. In the above example, the entry `params["c"]`
is a 3-dimensional `pandas.Series`. Thus, the second derivative output contains the
corresponding 3x3-block of the hessian at the position `["c"]["c"]`:

In [ ]:
sd.derivative["c"]["c"].round(3)

## There are many options

You can choose which finite difference method to use, whether we should respect
parameter bounds, or whether to evaluate the function in parallel. Let's go through
some basic examples. 

## You can choose the difference method

> Note. A mathematical explanation of the background of the difference methods can be
> found on the corresponding [explanation page](../explanation/numdiff_background.md).

In [ ]:
fd = om.first_derivative(
    func=fun,
    params=np.arange(5),
    method="backward",  # default: 'central'
)

fd.derivative

In [ ]:
sd = om.second_derivative(
    func=fun,
    params=np.arange(5),
    method="forward",  # default: 'central_cross'
)

sd.derivative.round(3)

## You can add bounds  

In [ ]:
params = np.arange(5)

fd = om.first_derivative(
    func=fun,
    params=params,
    # forces first_derivative to use forward differences
    bounds=om.Bounds(lower=params, upper=params + 1),
)

fd.derivative

Of course, bounds also work in `second_derivative`.